In [1]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [2]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [3]:
# Read in the data
train = list(read_dataset("../data/classes/train.txt"))
# 第２引数のw2iは普通のdictのコンストラクタに与えられる
# これ以降登録されていない語にUNK(0)を返すようにする
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [4]:
train[0][1]

0

In [5]:
# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [6]:
# Define the model
W_sm = model.add_lookup_parameters((nwords, ntags)) # Word weights
b_sm = model.add_parameters((ntags))                # Softmax bias

In [11]:
# A function to calculate scores for one value
def calc_scores(words):
  dy.renew_cg()   # build new computation graph
  score = dy.esum([dy.lookup(W_sm, x) for x in words])  # dy.esum: sum, 
  b_sm_exp = dy.parameter(b_sm)  # dy.parameter: load tensor to computational graph
  return score + b_sm_exp

In [16]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    # dy.pick: Picking values from vector expressions
    # dy.pick(e1, k) <-> e1[k]
    # dy.pickneglogsoftmax: neglogsoftmax -> pick (in this case dy.pick(~, tag))
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=2.3897, time=0.35s
iter 0: test acc=0.2615
iter 1: train loss/sent=2.0018, time=0.33s
iter 1: test acc=0.2701
iter 2: train loss/sent=1.8110, time=0.32s
iter 2: test acc=0.2819
iter 3: train loss/sent=1.6794, time=0.31s
iter 3: test acc=0.2869
iter 4: train loss/sent=1.5726, time=0.31s
iter 4: test acc=0.2959
iter 5: train loss/sent=1.4860, time=0.38s
iter 5: test acc=0.3005
iter 6: train loss/sent=1.4112, time=0.33s
iter 6: test acc=0.3090
iter 7: train loss/sent=1.3454, time=0.35s
iter 7: test acc=0.3077
iter 8: train loss/sent=1.2875, time=0.32s
iter 8: test acc=0.3145
iter 9: train loss/sent=1.2350, time=0.32s
iter 9: test acc=0.3190
iter 10: train loss/sent=1.1876, time=0.32s
iter 10: test acc=0.3217
iter 11: train loss/sent=1.1446, time=0.32s
iter 11: test acc=0.3294
iter 12: train loss/sent=1.1050, time=0.32s
iter 12: test acc=0.3222
iter 13: train loss/sent=1.0680, time=0.32s
iter 13: test acc=0.3321
iter 14: train loss/sent=1.0345, time=0.38s
iter 14: t